# Vertex AI AutoML Object Detection - Show Batch Prediction Results (from JSONL files)

Contributor: michaelmenzel@

In [ ]:
#@title Parameters { display-mode: "form" }
#@markdown Point to the bucket (and folder) with JSONL results from a batch prediction:
BUCKET = 'gs://license-plate-detection-us-central1/batch_predictions/prediction-license-plate-detection-us-central1_202161765327-2022-04-29T12:07:14.425828Z' #@param {type:"string"}

#@markdown How many of the batch prediction results do we process?
MAX_RESULTS =  100#@param {type:"integer", min:1}
#@markdown How many of the batch prediction results should be skipped?
SKIP_RESULTS =  0#@param {type:"integer", min:1}

#@markdown What is the min. confidence level to trust the model prediction (detecting a defect)?
THRESHOLD = 0.25 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown How many detected defects do we render on an image max?
TOP_K = 5 #@param {type:"integer", min:1}

#@markdown Do we output images with defects during the processing (in the notebook)?
SHOW_EXAMPLES = True #@param {type:"boolean"}

#@markdown [OPTIONAL] We (temporarily) store generated images in the following local folder:
OUTPUT_FOLDER = './test_output' #@param {type:"string"}
#@markdown [OPTIONAL] Font size of the annotation text.
FONTSIZE = 12 #@param {type:"integer", min:1}


import json
import os
from joblib import Parallel, delayed
from tqdm import tqdm

import tensorflow as tf
import numpy as np

from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.rcParams["figure.figsize"] = (20,20)

import googleapiclient.discovery
from google.oauth2 import service_account

from google.colab import auth
from google.colab import files


!mkdir -p $OUTPUT_FOLDER
auth.authenticate_user()


def plot_prediction(image, bboxes, classes, confidences, show=False):
    with tf.io.gfile.GFile(image, 'rb') as ex_img:
        img_tensor = tf.io.decode_image(ex_img.read())
        max_width = img_tensor.shape[1]
        max_height = img_tensor.shape[0]

        fig, ax = plt.subplots()
        ax.imshow(img_tensor)
        
        for bbox, cl, co in zip(bboxes, classes, confidences):
            x1 = max_width * np.min((bbox[0], bbox[1]))
            x2 = max_width * np.max((bbox[0], bbox[1]))
            y1 = max_height * np.min((bbox[2], bbox[3]))
            y2 = max_height * np.max((bbox[2], bbox[3]))

            color = cm.get_cmap('RdYlGn')
            ax.add_patch(patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                           linewidth=2, 
                                           edgecolor=color(co), fill=False))
            ax.text(x1, y2, f'{cl}@{format(co, ".5f")}', color=color(co), fontsize=FONTSIZE, verticalalignment='top')
            ax.set_axis_off()
            
            plt.savefig(os.path.join(OUTPUT_FOLDER, os.path.basename(image) + '.png'))
            
        plt.show() if show else plt.close(fig)


def process_result(result):
  threshold_filter = np.array(result['prediction']['confidences']) > THRESHOLD
  confidences = np.array(result['prediction']['confidences'])[threshold_filter]
  top_k = min(TOP_K, len(confidences))
  top_k_pos = np.argpartition(confidences, -top_k)[-top_k:]
  classes = np.array(result['prediction']['displayNames'])[top_k_pos]
  bboxes = np.array(result['prediction']['bboxes'])[top_k_pos]
    
  print('image:', result['instance']['content'], ', max confidences:', confidences, 'classes:', classes, 'bboxes:', bboxes)
  plot_prediction(result['instance']['content'], 
                  np.array(result['prediction']['bboxes'])[top_k_pos],
                  np.array(result['prediction']['displayNames'])[top_k_pos],
                  np.array(result['prediction']['confidences'])[top_k_pos],
                  show=(len(confidences) & SHOW_EXAMPLES))


print('Loading batch prediction results...')
results = []
for file in tqdm(tf.io.gfile.glob(os.path.join(BUCKET, f'*.jsonl'))):
    with tf.io.gfile.GFile(file, mode="r") as gf:
        for line in gf.readlines():
            results.append(json.loads(line))

print('Annotating Images...')
Parallel(n_jobs=8, backend='multiprocessing')(delayed(process_result)(result) for result in tqdm(results[SKIP_RESULTS:MAX_RESULTS]))
    
print('Zipping and downloading annotated images...')
!zip -r -qq results.zip $OUTPUT_FOLDER 
files.download('results.zip')